In [1]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback
import random
i=0
rand_mv = False

In [2]:
# Launch the JVM
jpype.startJVM(classpath=['./maze-server-v2019.4.jar'])

In [3]:
# import the Java modules
from java.io import IOException
from java.net import Socket
from java.net import UnknownHostException

from javax.net.ssl import SSLSocket
from javax.net.ssl import SSLSocketFactory
from javax.xml.bind import JAXBException
from javax.xml.bind import UnmarshalException

from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory
from de.fhac.mazenet.server.networking import XmlInputStream
from de.fhac.mazenet.server.networking import XmlOutputStream
from de.fhac.mazenet.server.game import Board
from de.fhac.mazenet.server.game import Position
from de.fhac.mazenet.server.game import Card

In [4]:
class Client:
    __id = None
    def __init__(self, name, address, port, tls):
        if(tls):
            self.sslSocketFactory = SSLSocketFactory.getDefault()
            self.sslSocket = self.sslSocketFactory.createSocket(address, port)
            self.in_ = XmlInputStream(self.sslSocket.getInputStream())
            self.out = XmlOutputStream(self.sslSocket.getOutputStream())
        else:
            self.socket = Socket(address, port)
            self.in_ = XmlInputStream(self.socket.getInputStream())
            self.out = XmlOutputStream(self.socket.getOutputStream())

        self.name = name
        self.instance = None
        self.address = address
        self.port = port
        self.tls = tls
    
    def getId(self):
        return int(self.__id)
    
    def resetId(self):
        self.__id = 0

    @staticmethod
    def getInstance(self, name, address, port, tls):
        if(self.instance == None): self.instance = Client(name, address, port, tls)
        return self.instance

    def login(self):
        print("the login has been done")
        login = MazeComMessageFactory.createLoginMessage(self.name)
        self.out.write(login)
        loginResponse = self.in_.readMazeCom()
        if(loginResponse.getMessagetype() == MazeComMessagetype.LOGINREPLY):
            self.__id = loginResponse.getLoginReplyMessage().getNewID()
        elif(loginResponse.getMessagetype() == MazeComMessagetype.ACCEPT):
            print("Login Fehler: Falsche Nachricht")
            sys.exit(1)
        elif (loginResponse.getMessagetype() == MazeComMessagetype.DISCONNECT):
            print("Login Fehler: Loginversuche")
            sys.exit(1)
        else:
            print("Login Fehler: bekommte Message Type" + loginResponse.getMessagetype())
            sys.exit(1)

    def play(self):
        while(True):
            try:
                receivedMazeCom = self.in_.readMazeCom()
                if(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                    self.awaitMove(receivedMazeCom)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                    self.accept(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                    print("You have been disconnected")
                    self.disconnect(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                    print("in MoveInfo")
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                    print("You have won")
                    sys.exit(0)
                else:
                    print("Unknown message type: " + receivedMazeCom.getMessagetype())
            except Exception:
                print(traceback.format_exc())

    def accept(self, errortype):
        if(errortype ==  Errortype.NOERROR):
            print("Alles Ok, Bewegung erlaubt.")
        elif(errortype ==  Errortype.AWAIT_MOVE):
            print("Falsche Nachricht, Bewegung nicht erlaubt.")
        elif(errortype ==  Errortype.ILLEGAL_MOVE):
            print("Regelverstoß, unerlaubte Bewegung.")
        else:
            print("Unbekannte Fehlermeldung: " + errortype)
    
    def disconnect(self, errortype):
        if(errortype ==  Errortype.TOO_MANY_TRIES):
            print("Zu viele Versuche. Geworfen vom Server.")
            sys.exit(1)
        else:
            print("Ende des Spiels.")
            sys.exit(0)




    def Move(self, awaitMove):
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)

        playerPosition = board.findPlayer(self.__id)
        ls_range = np.arange(1,6,2)
        potentialShiftMoves =  []
        for i in ls_range:
            potentialShiftMoves.append( Position(0, i))
            potentialShiftMoves.append( Position(6, i))
            potentialShiftMoves.append( Position(i, 0))
            potentialShiftMoves.append( Position(i, 6))
        
        print("forbidden:", board.getForbidden())
        
        try:
            potentialShiftMoves.remove(board.getForbidden())
        except Exception:
            print("element not in list. skipping")
        potentialMoves =  []
        for position in potentialShiftMoves:
            pos = position
            print(position)
            orientedShiftCards = self.moeglicheOrientierungen(board.getShiftCard())
            for orientedShiftCard in orientedShiftCards:
                print(pos)
                potentialMove =  MoveMessageData()
                potentialMove.setShiftPosition(position)
                potentialMove.setShiftCard(orientedShiftCard)
                potentialMove.setNewPinPos(playerPosition)
                potentialMoves.append(potentialMove)
                boardNext = board.fakeShift(potentialMove)
                print("player id:", self.__id)
                print("board player:", boardNext.findPlayer(self.__id))
                if boardNext.findPlayer(self.__id) != None:
                    
                    reachablePositions = boardNext.getAllReachablePositions(boardNext.findPlayer(self.__id))
                    treasurePositionData = boardNext.findTreasure(treasure)
                    if(treasurePositionData == None): continue
                    treasurePosition =  Position(treasurePositionData)
                    if(treasurePosition in reachablePositions):
                        potentialMove.setNewPinPos(treasurePosition)
                        return potentialMove
                    
        #Fall 2
        return potentialMoves[random.randint(0, len(potentialMoves)-1)]
        

    def moeglicheOrientierungen(self, shiftCard):
        shiftCards =  []
        shiftCards.append(shiftCard)

        card =  Card(shiftCard)
        openings = card.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            card2 =  Card(card)
            shiftCards.append(card2)
        else:
            if(card.getOrientation() != Card.Orientation.D0):
                shiftCards.append(Card(card.getShape(), Card.Orientation.D0, card.getTreasure()))
            if(card.getOrientation() != Card.Orientation.D90):
                shiftCards.append(Card(card.getShape(), Card.Orientation.D90, card.getTreasure()))
            if(card.getOrientation() != Card.Orientation.D180):
                shiftCards.append(Card(card.getShape(), Card.Orientation.D180, card.getTreasure()))
            if(card.getOrientation() != Card.Orientation.D270):
                shiftCards.append(Card(card.getShape(), Card.Orientation.D270, card.getTreasure()))
        
        return shiftCards
    
    def awaitMove(self, receivedMazeCom):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move = self.Move(awaitMove)
        mazeComToSend =  MazeCom()
        mazeComToSend.setId(self.__id)
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        print("move:", move)
        self.out.write(mazeComToSend)

In [5]:
HOST = "127.0.0.1"
PORT = 5123

In [6]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [7]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [8]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [9]:
ls_range = np.arange(1,6,2)
potentialShiftMoves =  []
for i in ls_range:
    potentialShiftMoves.append((0, i))
    potentialShiftMoves.append((6, i))
    potentialShiftMoves.append((i, 0))
    potentialShiftMoves.append((i, 6))

In [10]:
potentialShiftMoves

[(0, 1),
 (6, 1),
 (1, 0),
 (1, 6),
 (0, 3),
 (6, 3),
 (3, 0),
 (3, 6),
 (0, 5),
 (6, 5),
 (5, 0),
 (5, 6)]

In [11]:
class MazeEnv(Env):
    def __init__(self):
        super().__init__()
        # Actions we can take, rotate card left, rotate card right, push card, go up, down, left, right, no op
        # actions: push card, move, no op
        # self.action_space = Discrete(3)
        # actions: pick x and y where the next card should be pushed and rotation
        self.action_space = Box(low=np.array([1,1,0]), high=np.array([5,5,3]), dtype=np.int8)
        #  array
        #self.observation_space = Tuple((
        #                             Discrete(50),
        #                             Discrete(24),
        #                             Discrete(2))
        #                            )
        self.observation_space = Box(low=0, high=255, shape=(1,800,1100), dtype=np.uint8)
        self.cap = mss()
        self.game_location = {'top':200, 'left':100, 'width':1700, 'height':800}
        self.end_location = {'top':50, 'left':400, 'width':400, 'height':150}
        self.client = 0
    """    
        self.players = np.array([0,0,0,0])
        
    def add_player(self, client):
        for i in range(4):
            if self.players[i] == 0:
                self.players[i] = client
                break
    """
    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(card)
        openings = card_.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(card_)
        else:
            if(rot == 0):
                return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action, rand=False):
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)

        playerPosition = board.findPlayer(self.client.getId())
        potentialShiftMoves = []
        if(rand):
            position = Position(1, 6)
        else:
            position = Position(1, 6)
            
        potentialShiftMoves.append(position)
        print("forbidden:", board.getForbidden())
        
        try:
            potentialShiftMoves.remove(board.getForbidden())
        except Exception:
            print("element not in list. skipping")
        
        if len(potentialShiftMoves) == 0:
            print("forbidden move, reward decreased")
            return None
        else:
            orientedShiftCard = self.moeglicheOrientierungen(action[2], board.getShiftCard())
            potentialMove = MoveMessageData()
            potentialMove.setShiftPosition(position)
            potentialMove.setShiftCard(orientedShiftCard)
            potentialMove.setNewPinPos(playerPosition)
            boardNext = board.fakeShift(potentialMove)
            print("find player:", boardNext.findPlayer(self.client.getId()))
            reachablePositions = boardNext.getAllReachablePositions(boardNext.findPlayer(self.client.getId()))
            treasurePositionData = boardNext.findTreasure(treasure)
            print("treasure position data:", treasurePositionData)
            if(treasurePositionData == None):
                print("No treasure found. reward decreased")
                return None
            treasurePosition = Position(treasurePositionData)
            if(treasurePosition in reachablePositions):
                print("position found!")
                potentialMove.setNewPinPos(treasurePosition)
                return potentialMove
            
            print("Object not reachable: random move :/")
            return potentialMove
    
    def accept(self, errortype):
        print("hey there, its me the accept method")
        print("errortype:", errortype)
        print("The supposed errortype:", Errortype.NOERROR)
        if(errortype ==  Errortype.NOERROR):
            print("Alles Ok, Bewegung erlaubt.")
        elif(errortype ==  Errortype.AWAIT_MOVE):
            print("Falsche Nachricht, Bewegung nicht erlaubt.")
        elif(errortype ==  Errortype.ILLEGAL_MOVE):
            print("Regelverstoß, unerlaubte Bewegung.")
        else:
            print("Unbekannte Fehlermeldung: " + errortype)
    
    def disconnect(self, errortype):
        if(errortype ==  Errortype.TOO_MANY_TRIES):
            print("Zu viele Versuche. Geworfen vom Server.")
            sys.exit(1)
        else:
            print("Ende des Spiels.")
            sys.exit(0)

    def awaitMove(self, receivedMazeCom, action, rand=False):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move = self.Move(awaitMove, action, rand)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(self.client.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        self.client.out.write(mazeComToSend)
        return 10
    
    def step(self, action):
        print("-------------------------step start-------------------------")
        reward = 0
        time.sleep(2)
        try:
            print("------begin try-----")
            receivedMazeCom = self.client.in_.readMazeCom()
            print("the stupid receivedMazeCom:", receivedMazeCom.getMessagetype())
            print("it broke here")
            if(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                reward += self.awaitMove(receivedMazeCom, action)
                rand_mv = False
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE and rand_mv):
                reward += self.awaitMove(receivedMazeCom, action, rand_mv)
                rand_mv = False
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                self.client.accept(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                if(receivedMazeCom.getAcceptMessage().getErrortypeCode() == Errortype.ILLEGAL_MOVE):
                    print("bad move, reward decreased, random move chosen")
                    reward -= 10
                    rand_mv = True
                else:
                    print("accepted")
                    reward += 5
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                print("You have been disconnected")
                reward -= 10
                self.client.disconnect(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                print("in MoveInfo")
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                print("You have won")
                reward += 100
                sys.exit(0)
            else:
                print("Unknown message type: " + receivedMazeCom.getMessagetype())
            print("-----emnd try----")
        except Exception as e:
            #print(traceback.format_exc())
            #print("cut")
            if str(type(e)) == "<java class 'java.net.SocketException'>":
                print("socket error, disconnected")
                sys.exit(1)
            print("type e:", type(e))
            print(e)
        #############
        print("bfore the done :(")
        done, done_cap = self.get_done() 
        observation = self.get_observation()
        info = {}
        print("-------------------------step's over :)-------------------------")
        return observation, reward, done, info
        
    def render(self):
        # Implement viz
        plt.imshow(np.array(self.cap.grab(self.end_location)))
    
    def reset(self):
        print("reeeset time :]")
        while True:
            try:
                time.sleep(2)
                # stop
                pydirectinput.click(x=1550, y=200)
                # start
                pydirectinput.click(x=1450, y=200)
                time.sleep(2)
                self.client.login()
                break
            except:
                print("failed login, retrying :)")
                self.client = Client(self.client.name, self.client.address, self.client.port, self.client.tls)
            
        return self.get_observation()
    
    # Closes the game?
    def close(self):
        pass
    
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3]
        
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY) 
        
        resized = cv2.resize(gray, (100,83))
        channel = np.reshape(resized, (1,83,100))
        return channel
    
    def get_done(self):
        done_screen = np.array(self.cap.grab(self.end_location))
        
        done_strs = ['Winner', 'Winn']
        
        # ocr
        done = False
        psm = r'--psm 8'
        res = pytesseract.image_to_string(done_screen, config=psm)[:4]
        if res in done_strs:
            done = True
        print("done : ", done)
        return done, done_screen

In [12]:
menv = MazeEnv()

In [13]:
menv.client = Client('joe', HOST, PORT, False)

In [14]:
obs = menv.reset()
done = False  
total_reward   = 0
while not done:
    print("i = ", i)
    print(menv.action_space.sample())
    obs, reward,  done, info =  menv.step(menv.action_space.sample())
    #time.sleep(10)
    total_reward  += reward
    i += 1
print('Total Reward for episode {} is {}'.format(episode, total_reward))

reeeset time :]
the login has been done
Login Fehler: Loginversuche
failed login, retrying :)
the login has been done
Login Fehler: Loginversuche
failed login, retrying :)
the login has been done
i =  5
[5 1 2]
-------------------------step start-------------------------
------begin try-----
the stupid receivedMazeCom: AWAITMOVE
it broke here
forbidden: None
element not in list. skipping
find player: (0,0)
treasure position data: de.fhac.mazenet.server.generated.PositionData@6c4906d3
Object not reachable: random move :/
-----emnd try----
bfore the done :(
done :  False
-------------------------step's over :)-------------------------
i =  6
[1 5 2]
-------------------------step start-------------------------
------begin try-----
the stupid receivedMazeCom: ACCEPT
it broke here
Alles Ok, Bewegung erlaubt.
accepted
-----emnd try----
bfore the done :(
done :  False
-------------------------step's over :)-------------------------
i =  7
[2 5 3]
-------------------------step start-----------

AssertionError: 

In [ ]:
try:
    c = Client('joe', HOST, PORT, True)
    c.login()
    c.play()
except Exception:
    print(traceback.format_exc())

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(menv)